# Natural Language Processing

## Importing the libraries

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

## Importing the dataset

In [2]:
dataset = pd.read_csv('Ethos_Dataset_Binary.csv')

## Cleaning the texts

In [3]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 998):
  review = re.sub('[^a-zA-Z]', ' ', dataset['comment'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Creating the Bag of Words model

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2700)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values
for i in range(0,998):
  if y[i] >= 0.5:
    y[i] = int(1)
  else:
    y[i] = int(0)

## Splitting the dataset into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

## Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN

### Initialising the ANN

In [7]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [8]:
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))

### Adding the second hidden layer

In [9]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [10]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the ANN

### Compiling the ANN

In [11]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [12]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
24/24 [==============================] - 0s 2ms/step - loss: 0.7339 - accuracy: 0.5254
Epoch 2/100
24/24 [==============================] - 0s 2ms/step - loss: 0.5585 - accuracy: 0.7447
Epoch 3/100
24/24 [==============================] - 0s 2ms/step - loss: 0.4596 - accuracy: 0.8382
Epoch 4/100
24/24 [==============================] - 0s 2ms/step - loss: 0.3887 - accuracy: 0.8810
Epoch 5/100
24/24 [==============================] - 0s 2ms/step - loss: 0.3256 - accuracy: 0.9265
Epoch 6/100
24/24 [==============================] - 0s 2ms/step - loss: 0.2702 - accuracy: 0.9505
Epoch 7/100
24/24 [==============================] - 0s 2ms/step - loss: 0.2215 - accuracy: 0.9639
Epoch 8/100
24/24 [==============================] - 0s 2ms/step - loss: 0.1808 - accuracy: 0.9706
Epoch 9/100
24/24 [==============================] - 0s 2ms/step - loss: 0.1462 - accuracy: 0.9746
Epoch 10/100
24/24 [==============================] - 0s 2ms/step - loss: 0.1192 - accuracy: 0.9840
Epoch 11/

In [13]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

## Making the Confusion Matrix

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[58 92]
 [38 62]]


0.48

In [15]:
tp = cm[0][0]
tn = cm[1][1]
fp = cm[0][1]
fn = cm[1][0]
accuracy = (tp + tn)/(tp + tn + fp + fn)
precision = (tp)/(tp + fp)
recall = tp/(tp + fn)
f1_score = 2*precision*recall/(precision + recall)

In [16]:
print("Accuracy: ",accuracy)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1 Score: ",f1_score)

Accuracy:  0.48
Precision:  0.38666666666666666
Recall:  0.6041666666666666
F1 Score:  0.4715447154471545
